Модуль для вырезки и нормализации изображений найденных автомобильных номеров. Нейросеть обучена на изображениях формата .jpg размером 1280\*720 и 1920\*1080. Маски подготовлены с помощью программы LabelImg.

In [ ]:
# коллабные импорты
from google.colab import drive
from google.colab import files


drive.mount('/GD', force_remount=True)

!cp -r /GD/'My Drive'/'Colab Notebooks'/PlateRecognition/data.zip .
!cp -r /GD/'My Drive'/'Colab Notebooks'/PlateRecognition/masks_jpg.zip .
!cp -r /GD/'My Drive'/'Colab Notebooks'/PlateRecognition/masks_jpg_predict.zip .
!cp -r /GD/'My Drive'/'Colab Notebooks'/PlateRecognition/detection_result.csv .

!unzip data.zip
!unzip masks_jpg.zip
!unzip masks_jpg_predict.zip

!mkdir plates_jpg
!mkdir plates_jpg_predict

In [2]:
import os
import shutil
import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from random import choice, sample
from PIL import Image
from tqdm.notebook import tqdm
from sklearn.cluster import DBSCAN
from typing import Optional

In [3]:
df_result = pd.read_csv('detection_result.csv')
df_result.head(2)

,path_to_pics,path_to_true_mask,path_to_pred_mask,category,iou
0,pics_jpg/photo_45269@12-01-2022_01-57-11_0.jpg,masks_jpg/photo_45269@12-01-2022_01-57-11_0.jpg,masks_jpg_predict/photo_45269@12-01-2022_01-57...,train,NaN
1,pics_jpg/photo_44798@11-01-2022_16-35-50_360.jpg,masks_jpg/photo_44798@11-01-2022_16-35-50_360.jpg,masks_jpg_predict/photo_44798@11-01-2022_16-35...,train,0.671911


Предыдущий модуль передал нам маски-кляксы с содержащимися в них автомобильными номерами. Нужно решить 3 задачи:

- отделить маски друг от друга, если на одном фото несколько авто с номерами

- вырезать прямоугольники с номерами с каждой картинки

- нормализовать каждый прямоугольник

Сперва для работы будем использовать маски, размеченные человеком. На "нейросетевых" масках проверим работоспособность.

Для отделения кучек пикселей друг от друга, будем использовать кластеризацию DBScan. Применим проебразование Хафа для выделения главной линии (ожидается, что это будет горизонтальная линия номера). Далее, зная направление данной линии для каждого кластера, повернем номер в горизонтальное положение.

DBScan для кластеризации масок на отдельные номера

In [4]:
dbscan = DBSCAN(eps=25, min_samples=50)

Преобразование Хофа для поворота изображения в горизонатльное положение по главной линии Хофа

In [5]:
def hough_transform(image: np.ndarray,
                    edges: np.ndarray,
                    threshold: int=100) -> np.ndarray:
    """
    Преобразование Хофа для поворота изображения
    в горизонатльное положение по главной линии Хофа
    """
    while threshold > 0:
        lines = cv2.HoughLines(edges, 1, np.pi/180, threshold)

        if lines is not None:
            prime_line = lines[0][0]
            _, theta = prime_line

            height, width = image.shape[:2]
            center = (width // 2, height // 2)

            rotation_matrix = cv2 \
                .getRotationMatrix2D(center,
                                     np.degrees(theta - np.pi/2), 1.0)

            rotated_image = cv2 \
                .warpAffine(image,
                            rotation_matrix,
                             (width, height),
                            flags=cv2.INTER_LINEAR)

            return rotated_image

        threshold -= 5

    else:

        return image

Функция для обработки изображения

In [6]:
def image_transform(image_path  : str,
                    mask_path   : str,
                    save        : bool=False,
                    show        : bool=True,
                    dir_to_save : Optional[str]=None,) -> Optional[list[str]]:
    """
    Функция для сохранения изображений найденных автомобильных номеров
    Возвращает список с путями до созраненных изображений
    """
    image = cv2.imread(image_path)
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

    indices = np.argwhere(mask)
    pixels_array = np.array(indices)

    list_with_path = []

    if pixels_array.any():
        labels = dbscan.fit_predict(pixels_array)

        df = pd.DataFrame({'y': pixels_array[:, 0],
                           'x': pixels_array[:, 1],
                           'cluster': labels})

        for cluster in df['cluster'].unique():

            y_min = df[df['cluster'] == cluster]['y'].min()
            x_min = df[df['cluster'] == cluster]['x'].min()

            y_max = df[df['cluster'] == cluster]['y'].max()
            x_max = df[df['cluster'] == cluster]['x'].max()

            y_min = max(0, y_min - 2)
            x_min = max(0, x_min - 2)

            y_max = min(image.shape[0], y_max + 2)
            x_max = min(image.shape[1], x_max + 2)

            cropped_image = image[y_min:y_max, x_min:x_max]

            gray = cv2.cvtColor(cropped_image, cv2.COLOR_BGR2GRAY)
            edges = cv2.Canny(gray, 300, 600)
            result = hough_transform(cropped_image, edges)

            if show:
                fig, (ax1, ax2, ax3, ax4) = plt.subplots(nrows=1,
                                                        ncols=4,
                                                        figsize=(20, 3))

                ax1.imshow(image)
                ax1.axis('off')

                ax2.imshow(mask)
                ax2.axis('off')

                ax3.imshow(cropped_image)
                ax3.axis('off')

                ax4.imshow(result)
                ax4.axis('off')
                plt.show()

            if save and dir_to_save:
                path_to_save = image_path \
                    .replace('pics_jpg', dir_to_save) \
                    .replace('.jpg', f'_plate_{cluster}.jpg')
                list_with_path.append(path_to_save)
                cv2.imwrite(path_to_save, result)

    return list_with_path if list_with_path else None

In [7]:
for indx in tqdm(df_result.sample(5).index):

    image_path = df_result.loc[indx, 'path_to_pics']
    mask_path = df_result.loc[indx, 'path_to_true_mask']

    result = image_transform(image_path=image_path, mask_path=mask_path)

    print('*' * 25)

Output hidden; open in https://colab.research.google.com to view.

Проверим результат на масках, полученных от нейросети

In [8]:
for indx in tqdm(df_result.sample(5).index):

    image_path = df_result.loc[indx, 'path_to_pics']
    mask_path = df_result.loc[indx, 'path_to_pred_mask']

    result = image_transform(image_path=image_path, mask_path=mask_path)

    print('*' * 25)

Output hidden; open in https://colab.research.google.com to view.

Сохраним все нормализованные изображения номеров, полученные по "ручным" маскам и маскам от нейросети.

In [9]:
if not os.path.exists('normalized_hands_plates_jpg'):
    !mkdir normalized_hands_plates_jpg
    !mkdir normalized_nn_plates_jpg

Ручные маски

In [10]:
df_result['path_to_hand_plate'] = None
dir_to_save='normalized_hands_plates_jpg'

for indx in tqdm(df_result.index):

    image_path = df_result.loc[indx, 'path_to_pics']
    mask_path = df_result.loc[indx, 'path_to_true_mask']

    result = image_transform(image_path=image_path,
                             mask_path=mask_path,
                             save=True,
                             show=False,
                             dir_to_save=dir_to_save)
    if result:
        df_result.at[indx, 'path_to_hand_plate'] = result

  0%|          | 0/3755 [00:00<?, ?it/s]

Маски от нейросети

In [11]:
df_result['path_to_nn_plate'] = None
dir_to_save='normalized_nn_plates_jpg'

for indx in tqdm(df_result.index):

    image_path = df_result.loc[indx, 'path_to_pics']
    mask_path = df_result.loc[indx, 'path_to_pred_mask']

    result = image_transform(image_path=image_path,
                             mask_path=mask_path,
                             save=True,
                             show=False,
                             dir_to_save=dir_to_save)
    if result:
        df_result.at[indx, 'path_to_nn_plate'] = result

  0%|          | 0/3755 [00:00<?, ?it/s]

In [12]:
df_result.to_csv('detection_result_with_plates.csv', index=False)
shutil.make_archive('normalized_hands_plates_jpg', 'zip', '.', 'normalized_hands_plates_jpg')
shutil.make_archive('normalized_nn_plates_jpg', 'zip', '.', 'normalized_nn_plates_jpg')

!cp normalized_hands_plates_jpg.zip /GD/'My Drive'/'Colab Notebooks'/PlateRecognition/normalized_hands_plates_jpg.zip
!cp normalized_nn_plates_jpg.zip /GD/'My Drive'/'Colab Notebooks'/PlateRecognition/normalized_nn_plates_jpg.zip
!cp detection_result_with_plates.csv /GD/'My Drive'/'Colab Notebooks'/PlateRecognition/detection_result_with_plates.csv